# Slivka Client Demo

## Introduction

This notebook demonstrates how to use the Slivka Python client to interact with a Slivka server. You'll learn how to install the client, query available services, submit a job, and retrieve the results.

### Install Slivka Client

In [1]:
# %pip install slivka-client==1.2.1b1


### Configure Slivka Server URL

In [2]:
# Set the Slivka server URL
local_slivka_url = 'http://localhost:3203/'
# local_slivka_url = 'http://slivka-bio:8000/'  # For Slivka running in Docker
dundee_slivka_url = 'https://www.compbio.dundee.ac.uk/slivka/'

# Set to True to use the local Slivka server
use_local_slivka = True

slivka_server_url = local_slivka_url if use_local_slivka else dundee_slivka_url
print(f"Using Slivka server at: {slivka_server_url}")


Using Slivka server at: http://localhost:3203/


### Initialize Slivka Client

In [3]:
from slivka_client import SlivkaClient

client = SlivkaClient(slivka_server_url)
print(f"Connected to Slivka server at: {client.url}")


Connected to Slivka server at: http://localhost:3203/


/Users/smacgowan/Active/ligysis-drsasp-template-integration/LIGYSIS-web/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Query Available Services

In [4]:
# List available services
for service in client.services:
    print(f"Service name: {service.name}, ID: {service.id}")


Service name: IUPred, ID: iupred
Service name: Example Service, ID: example
Service name: GlobPlot, ID: globplot
Service name: JPred, ID: jpred
Service name: Jronn, ID: jronn
Service name: MAFFT, ID: mafft
Service name: MSAProbs, ID: msaprobs
Service name: MUSCLE, ID: muscle
Service name: ProbCons, ID: probcons
Service name: RNAalifold, ID: rnaalifold
Service name: TCoffee, ID: tcoffee
Service name: NoD, ID: nod
Service name: TPpred3, ID: tppred3
Service name: ColabFold, ID: colabfold
Service name: LIGYSIS, ID: ligysis
Service name: AACon, ID: aacon
Service name: ClustalO, ID: clustalo
Service name: ClustalW, ID: clustalw
Service name: DisEMBL, ID: disembl


### Select Ligysis Service


In [5]:
service = client['ligysis']
print(f"Selected service: {service.name} (ID: {service.id})")


Selected service: LIGYSIS (ID: ligysis)


### Submit a Job to FragSys


In [ ]:
# Define parameters for the job
data = {
    'uniprot_id': 'Q9UGL1',
    'format': 'mmcif',
    'variants': True,
    'override': True,
    'clust_method': 'average',
    'clust_dist': 0.5,
    'hmm_iters': 3
}

# Specify input directory or tarball
# input_dir_path = 'data/input_dir'
tarball_path = 'data/Q9UGL1_cif_IN_small.tar.gz'

# Option 1: Upload individual files
# from pathlib import Path
# files = {f"file_{i}": open(str(file), 'rb') for i, file in enumerate(Path(input_dir_path).glob('*'))}

# Option 2: Upload tarball
with open(tarball_path, 'rb') as tarball:
    files = {'input_dir': tarball}

    # Submit the job
    job = service.submit_job(data=data, files=files)
    print(f"Job submitted with ID: {job.id}")

Job submitted with ID: Z2QXJJxXXaZ63QLk


### Monitor Job Status


In [7]:
import time

# Poll the job status until it is completed
while job.status not in ('COMPLETED', 'FAILED'):
    print(f"Job status: {job.status}")
    time.sleep(10)

print(f"Job completed with status: {job.status}")


Job status: PENDING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job completed with status: COMPLETED


### Retrieve Job Results


In [8]:
import os

# Download each file in the job results
for file in job.files:
    download_path = f"./downloads/{file.id}"
    os.makedirs(os.path.dirname(download_path), exist_ok=True)
    file.dump(download_path)
    print(f"File {file.id} downloaded to {download_path}")

File Z2QXJJxXXaZ63QLk/OUT/input_structures/arpeggio/5fyu_updated.json downloaded to ./downloads/Z2QXJJxXXaZ63QLk/OUT/input_structures/arpeggio/5fyu_updated.json
File Z2QXJJxXXaZ63QLk/OUT/input_structures/arpeggio/5fy9_updated.json downloaded to ./downloads/Z2QXJJxXXaZ63QLk/OUT/input_structures/arpeggio/5fy9_updated.json
File Z2QXJJxXXaZ63QLk/OUT/input_structures/arpeggio/5fyu_updated_proc.pkl downloaded to ./downloads/Z2QXJJxXXaZ63QLk/OUT/input_structures/arpeggio/5fyu_updated_proc.pkl
File Z2QXJJxXXaZ63QLk/OUT/input_structures/arpeggio/5fy9_updated_proc.pkl downloaded to ./downloads/Z2QXJJxXXaZ63QLk/OUT/input_structures/arpeggio/5fy9_updated_proc.pkl
File Z2QXJJxXXaZ63QLk/OUT/input_structures/simple_cifs/input_structures_average_0.5.defattr downloaded to ./downloads/Z2QXJJxXXaZ63QLk/OUT/input_structures/simple_cifs/input_structures_average_0.5.defattr
File Z2QXJJxXXaZ63QLk/OUT/input_structures/simple_cifs/input_structures_average_0.5.cxc downloaded to ./downloads/Z2QXJJxXXaZ63QLk/OUT/